In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
def lmul_bits(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    #THIS IS THE VERSION THAT IS USED FOR THIS NOTEBOOK. it is based on the definition in the section 2 of the report
    """
    goal should be:
    two tensors of floats:
    Convert to bfloat16 bit representation
    Extract sign and field bits (exponent+mantissa)
    Add fields with bias compensation
    Handle overflow/underflow
    XOR signs
    Repack into BF16 bits and convert back to float32 for readability

    Except that we add like this bit shift at the end 
    result = result + (result / (1 << 5)) + (result / (1 << 6))
    because for some reason it's been off by like 5% (and this corrects that)
    you can remove that line if you want to measure the raw mult accuracy between torch tensors
      and write about that in the report (iirc it should be a further 10% accuracy drop though).
      It seems that for LSTM, because multiplication errors stack up a lot quicker the actual raw 
      mult. result should matter a lot more (as opposed to MLP which just cares about being in 
      ball park (relative) since its only like 2 mult operations)
    """
    #Convert to BF16 binary form (simulate using float32)
    a = a.to(torch.float32)
    b = b.to(torch.float32)
    #view raw bit patterns
    a_bits = a.view(torch.int32)
    b_bits = b.view(torch.int32)
    #Extract BF16 upper bits (simulate truncation)
    a_bf16 = (a_bits >> 16) & 0xFFFF
    b_bf16 = (b_bits >> 16) & 0xFFFF
    #Extract sign and field bits ---
    a_sign = (a_bf16 >> 15) & 0x1
    b_sign = (b_bf16 >> 15) & 0x1
    a_field = a_bf16 & 0x7FFF
    b_field = b_bf16 & 0x7FFF
    #Handle zeros / subnormals (exp == 0) ---
    a_exp = (a_field >> 7) & 0xFF
    b_exp = (b_field >> 7) & 0xFF
    zero_mask = (a_exp == 0) | (b_exp == 0)
    #Add the fields with bias correction ---
    OFFSET_MOD = 0x4080  # (2^15 - (127 << 7)) & 0x7FFF = 0x4080
    sum_full = a_field.to(torch.int32) + b_field.to(torch.int32) + OFFSET_MOD
    #use 17-bit precision
    carry2 = (sum_full >> 15) & 0x3  # top 2 bits
    field_sel = torch.zeros_like(sum_full)
    #Handle overflow/underflow
    # 00: underflow → 0
    mask_underflow = (carry2 == 0)
    # 01: normal → use lower 15 bits
    mask_normal = (carry2 == 1)
    # 1x: overflow → saturate to 0x7FFF
    mask_overflow = (carry2 >= 2)

    field_sel = torch.where(mask_normal, sum_full & 0x7FFF, field_sel)
    field_sel = torch.where(mask_overflow, torch.tensor(0x7FFF, dtype=torch.int32, device=sum_full.device), field_sel)

    #Calculate the sign bit
    s_result = (a_sign ^ b_sign).to(torch.int32)
    s_result = torch.where(field_sel == 0, torch.tensor(0, device=sum_full.device, dtype=torch.int32), s_result)
    #Pack result (sign << 15) | field
    result_bits_bf16 = ((s_result << 15) | field_sel).to(torch.int32)

    
    #Convert back to float32 by restoring 16 LSBs as zeros
    result_bits_f32 = result_bits_bf16 << 16
    result = result_bits_f32.view(torch.float32)
    #bias bitshift
    result = result + (result / (1 << 5)) + (result / (1 << 6))
    #Handle zeros
    result = torch.where(zero_mask, torch.zeros_like(result), result)
   
    return result 

## Quick note

In the lmul_bits defined above, there is a 1.05 multiplier (5%) bias applied at the end of the function.
This is to normalize the mean magnitude of L-Mul outputs to match true multiplication! 
(Note that its actually a bit shift that results in +4.69% which is close enough). 

# What about the LMUL in the rtl folder?

It takes ages to run a pure python non torch tensorized function in an LSTM (which is why lmul_bits was defined).

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=128, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=False, transform=transforms.ToTensor()),
    batch_size=1000
)

def lmul(a, b, M=7):
    #can add some internal logic here, but this is just a blank call to the lmul bits defined in the first cell
    return lmul_bits(a, b)

#similar logic to the MLP version
class LSTMLayer(nn.Module):
    def __init__(self, input_size, hidden_size, use_lmul=False, M=7):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.use_lmul = use_lmul
        self.M = M
        #Concatenated weight matrices (pretty sure this is the default way to do it)
        self.W = nn.Linear(input_size + hidden_size, 4 * hidden_size)

    def forward(self, x_t, h_prev, c_prev):
        combined = torch.cat((x_t, h_prev), dim=1)  # [B, I+H]

        if self.use_lmul:
            #manually replace the matmul in Linear with LMul IF enabled
            W = self.W.weight
            b = self.W.bias
            B, I = combined.shape
            O, _ = W.shape
            x_exp = combined.unsqueeze(1)   # [B, 1, I]
            W_exp = W.unsqueeze(0)          # [1, O, I]
            prod = lmul(x_exp, W_exp, M=self.M)
            gates = prod.sum(dim=2) + b
        else:
            gates = self.W(combined)

        i, f, g, o = torch.chunk(gates, 4, dim=1)
        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)

        if self.use_lmul:
            #cell update with LMul
            c_t = lmul(f, c_prev, M=self.M) + lmul(i, g, M=self.M)
            h_t = lmul(o, torch.tanh(c_t), M=self.M)
        else:
            c_t = f * c_prev + i * g
            h_t = o * torch.tanh(c_t)
        return h_t, c_t


class LSTMClassifier(nn.Module):
    def __init__(self, input_size=28, hidden_size=128, use_lmul=False, M=7):
        super().__init__()
        self.hidden_size = hidden_size
        self.use_lmul = use_lmul
        self.lstm_cell = LSTMLayer(input_size, hidden_size, use_lmul, M)
        self.fc = nn.Linear(hidden_size, 10)

    def forward(self, x):
        # x: [B, 1, 28, 28] (treat each row as a timestep)
        B = x.size(0)
        x = x.squeeze(1)  # [B, 28, 28]
        h = torch.zeros(B, self.hidden_size)
        c = torch.zeros(B, self.hidden_size)
        #this isn't the best way to do this; ideally you'd have some kind of feature extraction for 
        #the dim sizes but *if it works it works*
        for t in range(28):  #sequence length = 28
            x_t = x[:, t, :]  #each row = input vector
            h, c = self.lstm_cell(x_t, h, c)

        out = self.fc(h)
        return F.log_softmax(out, dim=1)

def train_model(model, optimizer, loader, epochs=2):
    model.train()
    for epoch in range(epochs):
        for data, target in loader:
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

def test_acc(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for data, target in loader:
            pred = model(data).argmax(dim=1)
            correct += (pred == target).sum().item()
            total += len(target)
    return 100 * correct / total

#(normal multiply)
model = LSTMClassifier(use_lmul=False)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
train_model(model, opt, train_loader, epochs=5)
baseline_acc = test_acc(model, test_loader)
print(f"Baseline LSTM accuracy: {baseline_acc:.2f}%")

#L-Mul arithmetic
model_lmul = LSTMClassifier(use_lmul=True, M=7)
model_lmul.load_state_dict(model.state_dict())
lmul_acc = test_acc(model_lmul, test_loader)
print(f"L-Mul LSTM accuracy: {lmul_acc:.2f}%")


Baseline LSTM accuracy: 97.49%
L-Mul LSTM accuracy: 97.71%


In [79]:
a = torch.tensor([5])
b = torch.tensor([5])
lmul(a, b)

tensor([25.1250])

In [77]:
a = torch.randn(10000000)
b = torch.randn(10000000)
prod = a * b
approx = lmul(a, b)
print("mean(|LMul|)/mean(|mul|) =", approx.abs().mean().item() / prod.abs().mean().item())

mean(|LMul|)/mean(|mul|) = 1.0016238620280844


## What is Fashion-MNIST?

Fashion-MNIST is a dataset of 28×28 grayscale images, just like MNIST digits, but instead of handwritten digits, each image is a clothing or accessory item from Zalando.

It was created because MNIST was too easy, as even our models can get high accuracy on MNIST, which isn’t useful for benchmarking anymore. Fashion-MNIST provides a similar format but more challenging, without being huge.

Let's see if there's an accuracy difference that appears! 

In [2]:
from torchvision import datasets, transforms
import torch

train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        '.', train=True, download=True,
        transform=transforms.ToTensor()
    ),
    batch_size=128, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        '.', train=False, download=True,
        transform=transforms.ToTensor()
    ),
    batch_size=128, shuffle=False
)

def lmul(a, b, M=7):
    #can add some internal logic here, but this is just a blank call to the lmul bits defined in the first cell
    return lmul_bits(a, b)

#similar logic to the MLP version
class LSTMLayer(nn.Module):
    def __init__(self, input_size, hidden_size, use_lmul=False, M=7):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.use_lmul = use_lmul
        self.M = M
        #Concatenated weight matrices (pretty sure this is the default way to do it)
        self.W = nn.Linear(input_size + hidden_size, 4 * hidden_size)

    def forward(self, x_t, h_prev, c_prev):
        combined = torch.cat((x_t, h_prev), dim=1)  # [B, I+H]

        if self.use_lmul:
            #manually replace the matmul in Linear with LMul IF enabled
            W = self.W.weight
            b = self.W.bias
            B, I = combined.shape
            O, _ = W.shape
            x_exp = combined.unsqueeze(1)   # [B, 1, I]
            W_exp = W.unsqueeze(0)          # [1, O, I]
            prod = lmul(x_exp, W_exp, M=self.M)
            gates = prod.sum(dim=2) + b
        else:
            gates = self.W(combined)

        i, f, g, o = torch.chunk(gates, 4, dim=1)
        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)

        if self.use_lmul:
            #cell update with LMul
            c_t = lmul(f, c_prev, M=self.M) + lmul(i, g, M=self.M)
            h_t = lmul(o, torch.tanh(c_t), M=self.M)
        else:
            c_t = f * c_prev + i * g
            h_t = o * torch.tanh(c_t)
        return h_t, c_t


class LSTMClassifier(nn.Module):
    def __init__(self, input_size=28, hidden_size=128, use_lmul=False, M=7):
        super().__init__()
        self.hidden_size = hidden_size
        self.use_lmul = use_lmul
        self.lstm_cell = LSTMLayer(input_size, hidden_size, use_lmul, M)
        self.fc = nn.Linear(hidden_size, 10)

    def forward(self, x):
        # x: [B, 1, 28, 28] (treat each row as a timestep)
        B = x.size(0)
        x = x.squeeze(1)  # [B, 28, 28]
        h = torch.zeros(B, self.hidden_size)
        c = torch.zeros(B, self.hidden_size)
        #this isn't the best way to do this; ideally you'd have some kind of feature extraction for 
        #the dim sizes but *if it works it works*
        for t in range(28):  #sequence length = 28
            x_t = x[:, t, :]  #each row = input vector
            h, c = self.lstm_cell(x_t, h, c)

        out = self.fc(h)
        return F.log_softmax(out, dim=1)

def train_model(model, optimizer, loader, epochs=2):
    model.train()
    for epoch in range(epochs):
        for data, target in loader:
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

def test_acc(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for data, target in loader:
            pred = model(data).argmax(dim=1)
            correct += (pred == target).sum().item()
            total += len(target)
    return 100 * correct / total

#(normal multiply)
model = LSTMClassifier(use_lmul=False)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
train_model(model, opt, train_loader, epochs=5)
baseline_acc = test_acc(model, test_loader)
print(f"Baseline LSTM accuracy: {baseline_acc:.2f}%")

#L-Mul arithmetic
model_lmul = LSTMClassifier(use_lmul=True, M=7)
model_lmul.load_state_dict(model.state_dict())
lmul_acc = test_acc(model_lmul, test_loader)
print(f"L-Mul LSTM accuracy: {lmul_acc:.2f}%")

100.0%
100.0%
100.0%
100.0%


Baseline LSTM accuracy: 86.39%
L-Mul LSTM accuracy: 84.97%


## Oh no!

Here we see a bit of an accuracy drop. This *is* to be expected, since LMUL is an approximate. Given that this is still one data point, however, let's do a test on other MNIST datasets.

Warning: The cell below takes a long time to run (about 15x longer than the previous)

In [3]:
import torch
from torchvision import datasets, transforms
import numpy as np
import random
import math
from collections import defaultdict

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#datasets mapping (function that returns train/test loaders); otherwise the code gets pretty long
def get_loaders(name, batch_train=128, batch_test=1000):
    T = transforms.ToTensor()
    if name == 'FashionMNIST':
        ds = datasets.FashionMNIST
    elif name == 'KMNIST':
        ds = datasets.KMNIST
    elif name == 'EMNIST_letters':
        ds = lambda root, train, download, transform: datasets.EMNIST(root, split='letters', train=train, download=download, transform=transform)
    elif name == 'MNIST':
        ds = datasets.MNIST
    elif name == 'SeqMNIST':  #this will use MNIST but will flatten in model 
        ds = datasets.MNIST
    elif name == 'CIFAR10_gray':
        #convert to grayscale
        T = transforms.Compose([transforms.Grayscale(), transforms.ToTensor()])
        ds = datasets.CIFAR10
    else:
        raise ValueError(name)

    train_loader = torch.utils.data.DataLoader(
        ds('.', train=True, download=True, transform=T),
        batch_size=batch_train, shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        ds('.', train=False, download=True, transform=T),
        batch_size=batch_test, shuffle=False
    )
    return train_loader, test_loader

#fix seeder
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

#Experiment runner
def run_experiment(datasets_list, seeds=[0,1,2,3,4], epochs=5, lr=1e-3):
    results = defaultdict(list)  
    #results[(dataset, condition)] -> list of accuracies

    for ds_name in datasets_list:
        train_loader, test_loader = get_loaders(ds_name)
        for seed in seeds:
            print(f"\n=== dataset={ds_name} seed={seed} ===")
            set_seed(seed)

            #training FP32 baseline model
            model_fp = LSTMClassifier(use_lmul=False).to(device)
            opt = torch.optim.Adam(model_fp.parameters(), lr=lr)
            train_model(model_fp, opt, train_loader, epochs=epochs)
            acc_fp = test_acc(model_fp, test_loader)
            print(f"FP32 acc: {acc_fp:.3f}")
            results[(ds_name, 'FP32')].append(acc_fp)

            #Eval same weights under LMUL inference
            model_lm = LSTMClassifier(use_lmul=True).to(device)
            model_lm.load_state_dict(model_fp.state_dict())  # identical weights
            acc_lm = test_acc(model_lm, test_loader)
            print(f"LMUL infer acc: {acc_lm:.3f}")
            results[(ds_name, 'LMUL_infer')].append(acc_lm)



    #basically give me the summ stats
    summary = {}
    for key, vals in results.items():
        arr = np.array(vals)
        summary[key] = (arr.mean(), arr.std(), len(arr))
    return summary, results

#u can select the other ones too but that would take too long for me to run ;;-;;
datasets_to_run = ['FashionMNIST', 'KMNIST', 'SeqMNIST'] 
summary, raw = run_experiment(datasets_to_run, seeds=[0,1,2,3,4], epochs=3, lr=1e-3)
print("\n|===>SUMMARY<===|")
for k, (mu, std, n) in summary.items():
    print(f"{k}: mean={mu:.2f} std={std:.2f} n={n}")



=== dataset=FashionMNIST seed=0 ===
FP32 acc: 84.610
LMUL infer acc: 83.550

=== dataset=FashionMNIST seed=1 ===
FP32 acc: 84.340
LMUL infer acc: 83.060

=== dataset=FashionMNIST seed=2 ===
FP32 acc: 84.200
LMUL infer acc: 83.910

=== dataset=FashionMNIST seed=3 ===
FP32 acc: 83.600
LMUL infer acc: 82.820

=== dataset=FashionMNIST seed=4 ===
FP32 acc: 84.730
LMUL infer acc: 84.010


100.0%
100.0%
100.0%
100.0%



=== dataset=KMNIST seed=0 ===
FP32 acc: 85.640
LMUL infer acc: 85.320

=== dataset=KMNIST seed=1 ===
FP32 acc: 85.530
LMUL infer acc: 85.460

=== dataset=KMNIST seed=2 ===
FP32 acc: 85.080
LMUL infer acc: 85.290

=== dataset=KMNIST seed=3 ===
FP32 acc: 86.480
LMUL infer acc: 86.490

=== dataset=KMNIST seed=4 ===
FP32 acc: 86.600
LMUL infer acc: 86.600

=== dataset=SeqMNIST seed=0 ===
FP32 acc: 96.610
LMUL infer acc: 96.400

=== dataset=SeqMNIST seed=1 ===
FP32 acc: 96.470
LMUL infer acc: 96.550

=== dataset=SeqMNIST seed=2 ===
FP32 acc: 96.090
LMUL infer acc: 96.180

=== dataset=SeqMNIST seed=3 ===
FP32 acc: 95.680
LMUL infer acc: 95.470

=== dataset=SeqMNIST seed=4 ===
FP32 acc: 96.960
LMUL infer acc: 96.920

|===>SUMMARY<===|
('FashionMNIST', 'FP32'): mean=84.30 std=0.40 n=5
('FashionMNIST', 'LMUL_infer'): mean=83.47 std=0.47 n=5
('KMNIST', 'FP32'): mean=85.87 std=0.58 n=5
('KMNIST', 'LMUL_infer'): mean=85.83 std=0.59 n=5
('SeqMNIST', 'FP32'): mean=96.36 std=0.44 n=5
('SeqMNIST', 'L

### What are these?

SeqMNIST is basically the MNIST dataset, but feeds the LSTM 1 pixel per step instead of a 28 dim vector. This means that the LSTM has to 'remember' or 'integrate' information from magnitudes more steps to come up with a classification as opposed to the normal dataset. 